In [1]:
import numpy as np
import cv2 as cv
import math

from PIL import Image
import torch
import torchvision
from torchvision.transforms import ToTensor

import datetime

import torch.quantization

In [3]:
# Load model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, progress=True, num_classes=91, 
                                                             pretrained_backbone=True, trainable_backbone_layers=0,
                                                            min_size=360)
model.eval();

## Quantization

In [29]:
# Dynamic quantization
model = torch.quantization.quantize_dynamic(model, {torch.nn.Conv2d}, dtype=torch.qint8)


In [35]:
# Dynamic quant
# set quantization config for server (x86)
#deploymentmyModel.qconfig = torch.quantization.get_default_config('fbgemm')

# insert observers
torch.quantization.prepare(model, inplace=True)
# Calibrate the model and collect statistics

# convert to quantized version
torch.quantization.convert(model, inplace=True)

/Users/David/anaconda3/lib/python3.7/site-packages/torch/quantization/quantize.py:170: UserWarning: None of the submodule got qconfig applied. Make sure you passed correct configuration through `qconfig_dict` or by assigning the `.qconfig` attribute directly on submodules
  warnings.warn("None of the submodule got qconfig applied. Make sure you "


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(360,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256)
          (relu): ReLU(inplace=True)
          (downsample)

## Data

In [4]:
## Data
path_in = ''
f = 'Intersection.jpg'

img = Image.open(path_in + f)
img = ToTensor()(img)

im_list = []
im_list.append(img)


In [6]:
begin_time = datetime.datetime.now()
print("Locating objects...")
model.eval()
predictions = model(im_list)

print(datetime.datetime.now() - begin_time)

Locating objects...
0:00:02.558974
